In [1]:
import os
os.listdir('../input/imet-2020-fgvc7')

['labels.csv', 'train.csv', 'train', 'test', 'sample_submission.csv']

In [2]:
import numpy as np
import pandas as pd
import random
import math

from collections import defaultdict, Counter
from pathlib import Path
from typing import Callable, List, Dict
from itertools import islice
import json
import shutil
import warnings
import cv2
from PIL import Image
from functools import partial

import torch
from torch.utils.data import Dataset, DataLoader
from torch.optim import Adam
from torchvision.transforms import (
    ToTensor, Normalize, Compose, Resize, CenterCrop, RandomCrop,
    RandomHorizontalFlip)
import torchvision.models as M

import tqdm
from sklearn.metrics import fbeta_score
from sklearn.exceptions import UndefinedMetricWarning


In [3]:
DATA_ROOT = '../input/imet-2020-fgvc7/'

In [4]:
def make_folds(n_folds: int):
    df = pd.read_csv(DATA_ROOT + 'train.csv')
    cls_counts = Counter(cls for classes in df['attribute_ids'].str.split()
                         for cls in classes)
    fold_cls_counts = defaultdict(int)
    folds = [-1] * len(df)
    for item in tqdm.tqdm(df.sample(frac=1, random_state=42).itertuples(),
                          total=len(df)):
        cls = min(item.attribute_ids.split(), key=lambda cls: cls_counts[cls])
        fold_counts = [(f, fold_cls_counts[f, cls]) for f in range(n_folds)]
        min_count = min([count for _, count in fold_counts])
        random.seed(item.Index)
        fold = random.choice([f for f, count in fold_counts
                              if count == min_count])
        folds[item.Index] = fold
        for cls in item.attribute_ids.split():
            fold_cls_counts[fold, cls] += 1
    df['fold'] = folds
    return df

In [5]:
df = make_folds(n_folds=5)
df.to_csv('folds.csv', index=None)

100%|██████████| 142119/142119 [00:03<00:00, 45287.67it/s]


In [6]:
df.head()

,id,attribute_ids,fold
0,000040d66f14ced4cdd18cd95d91800f,448 2429 782,2
1,0000ef13e37ef70412166725ec034a8a,2997 3231 2730 3294 3099 2017 784,0
2,0001eeb4a06e8daa7c6951bcd124c3c7,2436 1715 23,3
3,000226398d224de78b191e6db45fd94e,2997 3433 448 782,0
4,00029c3b0171158d63b1bbf803a7d750,3465 3322 3170 1553 781,0


In [7]:
N_CLASSES = 3474

In [8]:
def load_image(item, root):
    image = cv2.imread(str(root + '/' + f'{item.id}.png'))
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    return Image.fromarray(image)


def load_transform_image(item, root, image_transform, debug=False):
    image = load_image(item, root)
    image = image_transform(image)
    if debug:
        image.save('_debug.png')
    return tensor_transform(image)


def get_ids(root):
    return sorted({p.name.split('_')[0] for p in root.glob('*.png')})

In [9]:
class TrainDataset(Dataset):
    def __init__(self, root, df, image_transform, debug=True):
        super().__init__()
        self._root = root
        self._df = df
        self._image_transform = image_transform
        self._debug = debug

    def __len__(self):
        return len(self._df)

    def __getitem__(self, idx: int):
        item = self._df.iloc[idx]
        image = load_transform_image(item, self._root, 
                                     self._image_transform, debug=self._debug)
        target = torch.zeros(N_CLASSES)
        for cls in item.attribute_ids.split():
            target[int(cls)] = 1
        return image, target


class TTADataset:
    def __init__(self, root: Path, df: pd.DataFrame,
                 image_transform: Callable, tta: int):
        self._root = root
        self._df = df
        self._image_transform = image_transform
        self._tta = tta

    def __len__(self):
        return len(self._df) * self._tta

    def __getitem__(self, idx):
        item = self._df.iloc[idx % len(self._df)]
        image = load_transform_image(item, self._root, self._image_transform)
        return image, item.id

In [10]:
train_transform = Compose([
    RandomCrop(256),
    RandomHorizontalFlip(),
])

valid_transform = Compose([
    RandomCrop(256),
    RandomHorizontalFlip(),
])

tensor_transform = Compose([
    ToTensor(),
    Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

In [11]:
train_fold = df[df['fold'] != 0]
valid_fold = df[df['fold'] == 0]

In [12]:
train_root = DATA_ROOT + 'train'
num_workers = 4
batch_size = 64

In [13]:
def make_loader(df, image_transform):
    return DataLoader(
        TrainDataset(train_root, df, image_transform, debug=0),
        shuffle=True,
        batch_size=batch_size,
        num_workers=num_workers,
    )

In [14]:
train_fold.head()

,id,attribute_ids,fold
0,000040d66f14ced4cdd18cd95d91800f,448 2429 782,2
2,0001eeb4a06e8daa7c6951bcd124c3c7,2436 1715 23,3
5,0002f685f83528cc3c92ba7f01110db5,2883 3293 3465 3286 448 2635 2088 785,4
6,0002fe0e341a9563d0c01b9dab820222,3255 3127 2997 697 354 1060 1449 1354 784,4
7,00037f616b6937b239a7d9f2b5b9aa09,2902 448 2128 2103 784,2


In [15]:
train_loader = make_loader(train_fold, train_transform)
valid_loader = make_loader(valid_fold, valid_transform)
print(f'{len(train_loader.dataset):,} items in train, '
      f'{len(valid_loader.dataset):,} in valid')

113,694 items in train, 28,425 in valid


# Models

In [16]:
import torch.nn.functional as F

class AvgPool(torch.nn.Module):
    def forward(self, x):
        return F.avg_pool2d(x, x.shape[2:])


def create_net(net_cls, pretrained: bool):
    if pretrained:
        net = net_cls()
        model_name = net_cls.__name__
        weights_path = f'../input/{model_name}/{model_name}.pth'
        net.load_state_dict(torch.load(weights_path))
    else:
        net = net_cls(pretrained=pretrained)
    return net


class ResNet(torch.nn.Module):
    def __init__(self, num_classes, pretrained=False, 
                 net_cls=M.resnet50, dropout=False):
        super().__init__()
        self.net = create_net(net_cls, pretrained=pretrained)
        self.net.avgpool = AvgPool()
        if dropout:
            self.net.fc = torch.nn.Sequential(
                torch.nn.Dropout(),
                torch.nn.Linear(self.net.fc.in_features, num_classes),
            )
        else:
            self.net.fc = torch.nn.Linear(self.net.fc.in_features, num_classes)

    def fresh_params(self):
        return self.net.fc.parameters()

    def forward(self, x):
        return self.net(x)


resnet50 = partial(ResNet, net_cls=M.resnet50)
resnet101 = partial(ResNet, net_cls=M.resnet101)

# Train model

In [17]:
criterion = torch.nn.BCEWithLogitsLoss(reduction='none')

In [18]:
use_cuda = torch.cuda.is_available()
use_cuda

True

In [19]:
# model = resnet50(num_classes=N_CLASSES, pretrained=True, dropout=True)
model = resnet101(num_classes=N_CLASSES, pretrained=True)

In [20]:
model

ResNet(
  (net): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (downsample): Sequential(
          (0): 

In [21]:
fresh_params = list(model.fresh_params())
all_params = list(model.parameters())
if use_cuda:
    model = model.cuda()
for param in model.parameters():
    print(type(param.data), param.size())    

<class 'torch.Tensor'> torch.Size([64, 3, 7, 7])
<class 'torch.Tensor'> torch.Size([64])
<class 'torch.Tensor'> torch.Size([64])
<class 'torch.Tensor'> torch.Size([64, 64, 1, 1])
<class 'torch.Tensor'> torch.Size([64])
<class 'torch.Tensor'> torch.Size([64])
<class 'torch.Tensor'> torch.Size([64, 64, 3, 3])
<class 'torch.Tensor'> torch.Size([64])
<class 'torch.Tensor'> torch.Size([64])
<class 'torch.Tensor'> torch.Size([256, 64, 1, 1])
<class 'torch.Tensor'> torch.Size([256])
<class 'torch.Tensor'> torch.Size([256])
<class 'torch.Tensor'> torch.Size([256, 64, 1, 1])
<class 'torch.Tensor'> torch.Size([256])
<class 'torch.Tensor'> torch.Size([256])
<class 'torch.Tensor'> torch.Size([64, 256, 1, 1])
<class 'torch.Tensor'> torch.Size([64])
<class 'torch.Tensor'> torch.Size([64])
<class 'torch.Tensor'> torch.Size([64, 64, 3, 3])
<class 'torch.Tensor'> torch.Size([64])
<class 'torch.Tensor'> torch.Size([64])
<class 'torch.Tensor'> torch.Size([256, 64, 1, 1])
<class 'torch.Tensor'> torch.Size

In [22]:
def load_model(model, path):
    state = torch.load(str(path))
    model.load_state_dict(state['model'])
    print('Loaded model from epoch {epoch}, step {step:,}'.format(**state))
    return state

In [23]:
def _reduce_loss(loss):
    return loss.sum() / loss.shape[0]

In [24]:
def binarize_prediction(probabilities, threshold: float, argsorted=None,
                        min_labels=1, max_labels=10):
    """ Return matrix of 0/1 predictions, same shape as probabilities.
    """
    assert probabilities.shape[1] == N_CLASSES
    if argsorted is None:
        argsorted = probabilities.argsort(axis=1)
    max_mask = _make_mask(argsorted, max_labels)
    min_mask = _make_mask(argsorted, min_labels)
    prob_mask = probabilities > threshold
    return (max_mask & prob_mask) | min_mask


def _make_mask(argsorted, top_n: int):
    mask = np.zeros_like(argsorted, dtype=np.uint8)
    col_indices = argsorted[:, -top_n:].reshape(-1)
    row_indices = [i // top_n for i in range(len(col_indices))]
    mask[row_indices, col_indices] = 1
    return mask

In [25]:
def validation(model, criterion, valid_loader, use_cuda):
    model.eval()
    all_losses, all_predictions, all_targets = [], [], []
    with torch.no_grad():
        for inputs, targets in valid_loader:
            all_targets.append(targets.numpy().copy())
            if use_cuda:
                inputs, targets = inputs.cuda(), targets.cuda()
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            all_losses.append(_reduce_loss(loss).item())
            predictions = torch.sigmoid(outputs)
            all_predictions.append(predictions.cpu().numpy())
    all_predictions = np.concatenate(all_predictions)
    all_targets = np.concatenate(all_targets)

    def get_score(y_pred):
        with warnings.catch_warnings():
            warnings.simplefilter('ignore', category=UndefinedMetricWarning)
            return fbeta_score(all_targets, y_pred, beta=2, average='samples')

    metrics = {}
    argsorted = all_predictions.argsort(axis=1)
    for threshold in [0.10, 0.20]:
        metrics[f'valid_f2_th_{threshold:.2f}'] = get_score(
            binarize_prediction(all_predictions, threshold, argsorted))
    metrics['valid_loss'] = np.mean(all_losses)
    print(' | '.join(f'{k} {v:.3f}' for k, v in sorted(
        metrics.items(), key=lambda kv: -kv[1])))

    return metrics

In [26]:
train_kwargs = dict(
    model=model,
    criterion=criterion,
    train_loader=train_loader,
    valid_loader=valid_loader,
    patience=4,
    init_optimizer=lambda params, lr: Adam(params, lr),
    use_cuda=use_cuda,
)

In [27]:
model_path = '/kaggle/working/model.pt'
best_model_path = '/kaggle/working/my-best-model.pt'


def train(model, criterion, *, params, 
          train_loader, valid_loader, init_optimizer, use_cuda,
          n_epochs=None, patience=2, max_lr_changes=2):

    lr = 1e-4
    batch_size = 64
    n_epochs = 20
    params = list(params)
    optimizer = init_optimizer(params, lr)

    uptrain = False
    if uptrain:
        state = load_model(model, model_path)
        epoch = state['epoch']
        step = state['step']
        best_valid_loss = state['best_valid_loss']
        print(state)
    else:
        epoch = 1
        step = 0
        best_valid_loss = float('inf')
    lr_changes = 0

    save = lambda ep: torch.save({
        'model': model.state_dict(),
        'epoch': ep,
        'step': step,
        'best_valid_loss': best_valid_loss
    }, str(model_path))

    report_each = 100
    valid_losses = []
    lr_reset_epoch = epoch
    for epoch in range(epoch, n_epochs + 1):
        model.train()
        tq = tqdm.tqdm(total=(len(train_loader) * batch_size))
        tq.set_description(f'Epoch {epoch}, lr {lr}')
        losses = []
        tl = train_loader
        try:
            mean_loss = 0
            for i, (inputs, targets) in enumerate(tl):
                if use_cuda:
                    inputs, targets = inputs.cuda(), targets.cuda()
                outputs = model(inputs)
                loss = _reduce_loss(criterion(outputs, targets))
                batch_size = inputs.size(0)
                (batch_size * loss).backward()
                if (i + 1) % 1 == 0:
                    optimizer.step()
                    optimizer.zero_grad()
                    step += 1
                tq.update(batch_size)
                losses.append(loss.item())
                mean_loss = np.mean(losses[-report_each:])
                tq.set_postfix(loss=f'{mean_loss:.3f}')
            tq.close()
            save(epoch + 1)
            valid_metrics = validation(model, criterion, valid_loader, use_cuda)
            
            valid_loss = valid_metrics['valid_loss']
            valid_losses.append(valid_loss)
            if valid_loss < best_valid_loss:
                best_valid_loss = valid_loss
                shutil.copy(str(model_path), str(best_model_path))
            elif (patience and epoch - lr_reset_epoch > patience and
                  min(valid_losses[-patience:]) > best_valid_loss):
                lr_changes +=1
                if lr_changes > max_lr_changes:
                    break
                lr /= 5
                print(f'lr updated to {lr}')
                lr_reset_epoch = epoch
                optimizer = init_optimizer(params, lr)
        except KeyboardInterrupt:
            tq.close()
            print('Ctrl+C, saving snapshot')
            save(epoch)
            print('done.')
            return False
    return True

In [28]:
train(params=all_params, **train_kwargs)

Epoch 1, lr 0.0001: 100%|█████████▉| 113694/113728 [21:18<00:00, 88.95it/s, loss=16.629]


valid_loss 16.383 | valid_f2_th_0.10 0.462 | valid_f2_th_0.20 0.410


Epoch 2, lr 0.0001: : 113694it [21:13, 89.30it/s, loss=14.356]


valid_loss 14.078 | valid_f2_th_0.10 0.527 | valid_f2_th_0.20 0.494


Epoch 3, lr 0.0001: : 113694it [21:18, 88.93it/s, loss=12.973]


...

Epoch 20, lr 2e-05:   0%|          | 0/53310 [00:00<?, ?it/s]

valid_loss 10.952 | valid_f2_th_0.10 0.651 | valid_f2_th_0.20 0.635


Epoch 20, lr 2e-05: : 99008it [18:43, 90.00it/s, loss=5.333]

valid_loss 11.051 | valid_f2_th_0.10 0.650 | valid_f2_th_0.20 0.636


True